# Pengolahan Data Hasil Ukur

Software yang digunakan untuk pengukuran adalah Realtime Analyzer (paket dari DSSF3) versi 5.2.0.26 buatan Yoshimasha Electronic Inc.
Software tersebut running di atas layer Wine 6.13 dan PulseAudio 14.2 di sistem operasi Arch-Linux kernel 5.13.4.

Hasil pengukuran menggunakan FFT Analyzer pada 1/3 Octave Band dengan A-Weighting untuk membuang noise frekuensi rendah

In [1]:
# import necessary modules

%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

Rumus SPL Accumulation untuk 1/3 Octave band spectrums.

Didapat dari: https://www.cirrusresearch.co.uk/blog/2020/03/calculation-of-dba-from-octave-band-sound-pressure-levels/

In [2]:
def SplAll(dataIn):
    freqSum = 0
    for i in range(len(dataIn)):
        valeach = np.power(10,dataIn[i]/10)
        freqSum = freqSum + valeach
    freqAll = 10* np.log10(freqSum)
    
    return freqAll

Fungsi Data Process dengan batasan upper frekuensi tertentu.
Melihat analisa frekuensi dari audio rekaman yang digunakan, maka frekuensi di atas 12500 Hz tidak signifikan dan akan diabaikan.
Untuk itu, upFreq ditentukan sebagai 59 (L) dan 60 (R) sesuai index array rekaman.

In [3]:
def DataProcess(csvFile, lowFreq=4, upFreq=59):
    data = pd.read_csv(csvFile)
    
    allL = np.zeros(30)
    allR = np.zeros(30)
    
    avgL = np.zeros(30)
    avgR = np.zeros(30)

    for i in range(30):
        allL[i] = SplAll(data.iloc[i,lowFreq:upFreq:2])
        allR[i] = SplAll(data.iloc[i,lowFreq+1:upFreq+1:2])
    
    for i in range(30):
        avgL[i] = np.average(data.iloc[i,lowFreq:upFreq:2])
        avgR[i] = np.average(data.iloc[i,lowFreq+1:upFreq+1:2])
        
    print("%s || Sum L: %5.2f dB & R: %5.2f dB || Avg L: %5.2f dB & R: %5.2f dB" % (csvFile, np.average(allL), np.average(allR), np.average(avgL), np.average(avgR)))

Proses data CSV dari rekaman software DSSF3 Real-Time Analyzer di 1/3 Octave Band Analysis untuk speech

In [4]:
speech_35 = DataProcess('./speech_35.csv')
speech_40 = DataProcess('./speech_40.csv')
speech_45 = DataProcess('./speech_45.csv')
speech_50 = DataProcess('./speech_50.csv')
speech_55 = DataProcess('./speech_55.csv')
speech_60 = DataProcess('./speech_60.csv')

./speech_35.csv || Sum L: 59.56 dB & R: 59.78 dB || Avg L: 34.47 dB & R: 34.86 dB
./speech_40.csv || Sum L: 62.63 dB & R: 62.85 dB || Avg L: 36.17 dB & R: 36.66 dB
./speech_45.csv || Sum L: 65.54 dB & R: 65.84 dB || Avg L: 37.71 dB & R: 38.35 dB
./speech_50.csv || Sum L: 69.89 dB & R: 70.20 dB || Avg L: 40.57 dB & R: 41.34 dB
./speech_55.csv || Sum L: 75.70 dB & R: 75.97 dB || Avg L: 43.92 dB & R: 44.75 dB
./speech_60.csv || Sum L: 79.92 dB & R: 80.27 dB || Avg L: 47.04 dB & R: 47.93 dB


Proses data CSV dari rekaman software DSSF3 Real-Time Analyzer di 1/3 Octave Band Analysis untuk wishper

In [5]:
wishper_35 = DataProcess('./wishper_35.csv')
wishper_40 = DataProcess('./wishper_40.csv')
wishper_45 = DataProcess('./wishper_45.csv')
wishper_50 = DataProcess('./wishper_50.csv')
wishper_55 = DataProcess('./wishper_55.csv')
wishper_60 = DataProcess('./wishper_60.csv')

./wishper_35.csv || Sum L: 44.53 dB & R: 43.71 dB || Avg L: 25.22 dB & R: 25.05 dB
./wishper_40.csv || Sum L: 44.64 dB & R: 43.85 dB || Avg L: 25.16 dB & R: 24.94 dB
./wishper_45.csv || Sum L: 44.77 dB & R: 44.06 dB || Avg L: 25.17 dB & R: 24.93 dB
./wishper_50.csv || Sum L: 45.44 dB & R: 44.96 dB || Avg L: 25.41 dB & R: 25.18 dB
./wishper_55.csv || Sum L: 46.58 dB & R: 46.44 dB || Avg L: 25.68 dB & R: 25.62 dB
./wishper_60.csv || Sum L: 48.92 dB & R: 49.29 dB || Avg L: 26.89 dB & R: 27.17 dB


## Parceval's Theorem

Create array of absolute sum of each frequencies

In [6]:
def ParcevalCalc(csvFile, lowFreq=4, upFreq=59):
    
    # read data
    data = pd.read_csv(csvFile)
    
    # generate zero array
    sumL = np.zeros(np.shape(data.iloc[0,lowFreq:upFreq:2]))
    sumR = np.zeros(np.shape(data.iloc[0,lowFreq+1:upFreq+1:2]))
    
    # Left Part
    i = 0
    for j in range(lowFreq,upFreq,2):
        for t in range(30):
            sumL[i] = sumL[i] + np.abs(data.iloc[t,j])
        i = i + 1
    
    # Right Part
    i = 0
    for j in range(lowFreq+1,upFreq+1,2):
        for t in range(30):
            sumR[i] = sumR[i] + np.abs(data.iloc[t,j])
        i = i + 1
        
    # output as both array
    return np.vstack((sumL,sumR))

In [7]:
def NpToPd(dataIn):
    freqs = ['25','31.5','40','50','63','80','100','125','160','200','250','315','400','500','630','800','1K','1K25','1K6','2K','2K5','3K15','4K','5K','6K3','8K','10K','12K5']
    return pd.DataFrame(data=dataIn,columns=freqs, index=['L','R'])

In [8]:
pdData = NpToPd(ParcevalCalc('./speech_35.csv'))
pdData

,25,31.5,40,50,63,80,100,125,160,200,...,1K6,2K,2K5,3K15,4K,5K,6K3,8K,10K,12K5
L,43.04,375.43,569.12,762.81,635.02,393.00,833.41,1053.10,1005.26,1216.39,...,1099.20,1036.41,1091.40,1234.59,1112.72,1050.89,1036.96,1037.26,1028.31,1012.19
R,79.00,410.37,597.80,785.25,651.36,421.28,791.03,1076.91,1002.14,1193.59,...,1144.16,1093.45,1217.35,1293.74,1126.87,1027.66,990.61,995.53,988.87,970.88
